Load libraries

In [13]:
## For some reason, when loading mlogit, the notebook can't find package 'statmod' so I specify it's location
library(statmod, lib.loc='D:\\Applications\\Anaconda2\\pkgs\\r-statmod-1.4.30-r3.4.1_0\\lib\\R\\library\\')
require(mlogit)
require(ggplot2)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(car)
require(boot)
require(dplyr)
require(sjstats)
require(broom)

# Load data and set factors

In [14]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# sid is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$cvs_graph <- factor(mydata$cvs_graph)
mydata$cvs_table <- factor(mydata$cvs_table)

In [15]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163,L,Concentration,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,L,Width,0,2,1,1,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163,C,Area,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10127163,C,Separation,2,2,1,1,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10232160,L,Concentration,0,0,1,1,1,1,LC,...,1,1,2,1,1,1,1,1,1,4
10232160,L,Width,0,0,0,0,1,1,LC,...,1,1,0,1,1,1,1,1,1,4


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"

# Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)
* student attibutes (lab_experience, similar_sim, prior_number_virtual_labs)

We ignore attitude components.

## Predict main as a continous variable using an ANOVA

Some resources:
* On SS Types: https://mcfromnz.wordpress.com/2011/03/02/anova-type-iiiiii-ss-explained/
* on drop() function to do type 3: https://www.statmethods.net/stats/anova.html
* On repeated measures: http://psych.wisc.edu/moore/Rpdf/610-R8_OneWayWithin.pdf, https://datascienceplus.com/two-way-anova-with-repeated-measures/
* the car package: https://cran.r-project.org/web/packages/car/car.pdf

In [16]:
lm1 = lm(main
        ~  cvs_table + cvs_graph + variable + sim_index + pre + sid
         + lab_experience + similar_sim + prior_number_virtual_labs,
         data=mydata)
results1 = Anova(lm1, type=2)
results_table1 = tidy(results1)
results_table1$eta <- results_table1$sumsq/(results_table1$sumsq + results_table1$sumsq[dim(results_table1)[1]])
results_table1

Warning message:
"package 'bindrcpp' was built under R version 3.4.4"

term,sumsq,df,statistic,p.value,eta
cvs_table,2.948199e-03,1,0.012941292,9.094809e-01,2.988662e-05
cvs_graph,2.847763e+00,1,12.500420043,4.507331e-04,2.805928e-02
variable,2.053713e+00,3,3.004964024,3.020047e-02,2.039500e-02
sim_index,2.747183e+00,1,12.058916980,5.672906e-04,2.709510e-02
pre,5.046346e-01,1,2.215122837,1.373921e-01,5.089719e-03
sid,1.002275e+02,146,3.013387963,8.909223e-19,5.039834e-01
lab_experience,NA,0,NA,NA,NA
similar_sim,1.078127e-03,1,0.004732502,9.451860e-01,1.092945e-05
prior_number_virtual_labs,NA,0,NA,NA,NA
Residuals,9.864319e+01,433,NA,NA,5.000000e-01


We see that, in order of significance: cvs_graph, sim_index, and variable matter.

## Predict main as a categorical variable using multinomial logistic regression

In [18]:
mydata$main <- factor(mydata$main)

In [19]:
# data("Train", package = "mlogit")
# Tr <- mlogit.data(Train, shape = "wide", varying = 4:11,
# choice = "choice", sep = "_",
# opposite = c("price", "time", "change", "comfort"),
# alt.levels=c("A", "B"), id.var ="id")
# Train.ml <- mlogit(choice ~ price + time + change + comfort, Tr)
# Train.mxlc <- mlogit(choice ~ price + time + change + comfort, Tr,
# panel = TRUE, rpar = c(time = "cn", change = "n", comfort = "ln"),
# correlation = TRUE, R = 100, halton = NA)
# Train.mxlu <- update(Train.mxlc, correlation = FALSE)


In [48]:
data("Fishing", package = "mlogit")
Fish <- mlogit.data(Fishing, varying = c(2:9), shape = "wide", choice = "mode")
Fishing
Fish

mode,price.beach,price.pier,price.boat,price.charter,catch.beach,catch.pier,catch.boat,catch.charter,income
charter,157.930,157.930,157.930,182.930,0.0678,0.0503,0.2601,0.5391,7083.332
charter,15.114,15.114,10.534,34.534,0.1049,0.0451,0.1574,0.4671,1250.000
boat,161.874,161.874,24.334,59.334,0.5333,0.4522,0.2413,1.0266,3750.000
pier,15.134,15.134,55.930,84.930,0.0678,0.0789,0.1643,0.5391,2083.333
boat,106.930,106.930,41.514,71.014,0.0678,0.0503,0.1082,0.3240,4583.332
charter,192.474,192.474,28.934,63.934,0.5333,0.4522,0.1665,0.3975,4583.332
beach,51.934,51.934,191.930,220.930,0.0678,0.0789,0.1643,0.5391,8750.001
charter,15.134,15.134,21.714,56.714,0.0678,0.0789,0.0102,0.0209,2083.333
boat,34.914,34.914,34.914,53.414,0.2537,0.1498,0.0233,0.0219,3750.000
boat,28.314,28.314,28.314,46.814,0.2537,0.1498,0.0233,0.0219,2916.667


,mode,income,alt,price,catch,chid
1.beach,FALSE,7083.332,beach,157.930,0.0678,1
1.boat,FALSE,7083.332,boat,157.930,0.2601,1
1.charter,TRUE,7083.332,charter,182.930,0.5391,1
1.pier,FALSE,7083.332,pier,157.930,0.0503,1
2.beach,FALSE,1250.000,beach,15.114,0.1049,2
2.boat,FALSE,1250.000,boat,10.534,0.1574,2
2.charter,TRUE,1250.000,charter,34.534,0.4671,2
2.pier,FALSE,1250.000,pier,15.114,0.0451,2
3.beach,FALSE,3750.000,beach,161.874,0.5333,3
3.boat,TRUE,3750.000,boat,24.334,0.2413,3


In [51]:
m <- mlogit(mode ~ price + catch | income, data = Fish)
summary(m)


Call:
mlogit(formula = mode ~ price + catch | income, data = Fish, 
    method = "nr", print.level = 0)

Frequencies of alternatives:
  beach    boat charter    pier 
0.11337 0.35364 0.38240 0.15059 

nr method
7 iterations, 0h:0m:0s 
g'(-H)^-1g = 1.37E-05 
successive function values within tolerance limits 

Coefficients :
                       Estimate  Std. Error  z-value  Pr(>|z|)    
boat:(intercept)     5.2728e-01  2.2279e-01   2.3667 0.0179485 *  
charter:(intercept)  1.6944e+00  2.2405e-01   7.5624 3.952e-14 ***
pier:(intercept)     7.7796e-01  2.2049e-01   3.5283 0.0004183 ***
price               -2.5117e-02  1.7317e-03 -14.5042 < 2.2e-16 ***
catch                3.5778e-01  1.0977e-01   3.2593 0.0011170 ** 
boat:income          8.9440e-05  5.0067e-05   1.7864 0.0740345 .  
charter:income      -3.3292e-05  5.0341e-05  -0.6613 0.5084031    
pier:income         -1.2758e-04  5.0640e-05  -2.5193 0.0117582 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Lo

In [63]:
dt <- mlogit.data(mydata, shape='long',
    choice = "choice",
    opposite = c("cvs_table","cvs_graph","variable","sim_index"),
    alt.var = "variable", chid.var='sid')
head(dt)

Warning message in Ops.factor(data[[i]]):
"'-' not meaningful for factors"Warning message in Ops.factor(data[[i]]):
"'-' not meaningful for factors"Warning message in Ops.factor(data[[i]]):
"'-' not meaningful for factors"Warning message in Ops.factor(data[[i]]):
"'-' not meaningful for factors"

,sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,pre_with_ident,main_with_ident,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars
10127163.Concentration,10127163,L,NA,0,2,NA,NA,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163.Width,10127163,L,NA,0,2,NA,NA,1,1,LC,...,1,3,2,1,1,1,1,1,1,4
10127163.Area,10127163,C,NA,2,2,NA,NA,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10127163.Separation,10127163,C,NA,2,2,NA,NA,1,1,LC,...,3,3,2,1,1,1,1,1,1,4
10232160.Concentration,10232160,L,NA,0,0,NA,NA,1,1,LC,...,1,1,2,1,1,1,1,1,1,4
10232160.Width,10232160,L,NA,0,0,NA,NA,1,1,LC,...,1,1,0,1,1,1,1,1,1,4


In [66]:
# multimixed1.ml <= mlogit(main ~ cvs_table + cvs_graph + variable + sim_index + pre
#     + lab_experience + similar_sim + prior_number_virtual_labs, dt)
multimixed1.mlc <- mlogit(main ~ variable + sim_index + pre, dt)
#     + lab_experience + similar_sim + prior_number_virtual_labs, dt,

summary(multimixed1.mlc)
# summary(multimixed1)$coefficients

ERROR: Error in tological(mf[[1]]): the number of levels for the choice variable should equal two


In [19]:
multimixed1 <- multinom(
    main
    ~ cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs, #+ (1 | sid),
           data = mydata) #, family = binomial, 
           #control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)
summary(multimixed1)
summary(multimixed1)$coefficients

# weights:  36 (22 variable)
initial  value 645.984026 
iter  10 value 509.145932
iter  20 value 481.880642
final  value 481.199621 
converged


Call:
multinom(formula = main ~ cvs_table + cvs_graph + variable + 
    sim_index + pre + lab_experience + similar_sim + prior_number_virtual_labs, 
    data = mydata)

Coefficients:
  (Intercept) cvs_table1 cvs_graph1 variableConcentration variableSeparation
1  1.74682726 -0.2189315 0.05716703            -0.1795551         -0.3450247
2  0.08557114  0.4246402 1.23281280             0.1417915         -0.6937682
  variableWidth sim_index2       pre lab_experience1 similar_sim1
1   -0.34845656  0.1041758 0.4025732      -0.3018661   -0.6189754
2    0.07941073  0.6053822 1.0319006      -0.4265037   -0.4781605
  prior_number_virtual_labs
1                 0.2840173
2                 0.2717262

Std. Errors:
  (Intercept) cvs_table1 cvs_graph1 variableConcentration variableSeparation
1   0.5660631  0.4003265  0.4377776             0.4609949          0.4265563
2   0.6207209  0.4418659  0.4538844             0.4807653          0.4581335
  variableWidth sim_index2       pre lab_experience1 simila

,(Intercept),cvs_table1,cvs_graph1,variableConcentration,variableSeparation,variableWidth,sim_index2,pre,lab_experience1,similar_sim1,prior_number_virtual_labs
1,1.74682726,-0.2189315,0.05716703,-0.1795551,-0.3450247,-0.34845656,0.1041758,0.4025732,-0.3018661,-0.6189754,0.2840173
2,0.08557114,0.4246402,1.23281280,0.1417915,-0.6937682,0.07941073,0.6053822,1.0319006,-0.4265037,-0.4781605,0.2717262


Above are coefficients and standard erros. Here are p-values:

In [17]:
z <- summary(multimixed1)$coefficients/summary(multimixed1)$standard.errors
p <- (1 - pnorm(abs(z), 0, 1))*2
p

,(Intercept),cvs_table1,cvs_graph1,variableConcentration,variableSeparation,variableWidth,sim_index2,pre,lab_experience1,similar_sim1,prior_number_virtual_labs
1,0.002029213,0.5844595,0.896103893,0.6969102,0.4185952,0.4338588,0.72974158,0.22579333,0.5712789,0.09041525,0.1184847
2,0.890352889,0.3365441,0.006604836,0.7680483,0.1299405,0.8649764,0.05608372,0.00210292,0.4648508,0.21366441,0.1529842


# Stat model 2: Predicting transfer data

## Excluding student main worksheet score

In [7]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed1)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ cvs_table + cvs_graph + variable + sim_index +  
    pre + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   657.7    710.2   -316.8    633.7      576 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6284 -0.4560  0.2814  0.4788  1.3945 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.577    1.891   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)                0.58640    0.69050   0.849  0.39575   
cvs_table1                -0.24456    0.41942  -0.583  0.55983   
cvs_graph1                -0.13990    0.43279  -0.323  0.74651   
variableConcentration     -0.09

As expected, CVS doesn't predict quant transfer scores, only variable does.

## Including student main worksheet score
as a categorical variable

In [8]:
mydata$main <- factor(mydata$main)

In [9]:
mixed1 <- glmer(
    quant_score
    ~ cvs_table + cvs_graph + variable + sim_index + pre + main
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed1)


Correlation matrix not shown by default, as p = 13 > 12.
Use print(obj, correlation=TRUE)  or
	 vcov(obj)	 if you need it



Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: quant_score ~ cvs_table + cvs_graph + variable + sim_index +  
    pre + main + lab_experience + similar_sim + prior_number_virtual_labs +  
    (1 | sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   632.3    693.6   -302.2    604.3      574 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4348 -0.3919  0.2579  0.4439  2.3541 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 4.015    2.004   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -0.9854     0.8519  -1.157  0.24739    
cvs_table1                 -0.2844     0.4456  -0.638  0.52326    
cvs_graph1                 -0.5925     0.4770  -1.242  0.21422    
variableConcentratio

# Stat model 3: Predicting the use of CVS

## For cvs_table

In [6]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_table ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   636.9    693.8   -305.4    610.9      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7511 -0.4307  0.2088  0.3572  2.3885 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 5.991    2.448   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)               -1.20670    1.87139  -0.645  0.51905   
variableConcentration      0.53139    0.36369   1.461  0.14399   
variableSeparation        -0.25492    0.33945  -0.751  0.45267   
variableWidth              0.2

## For cvs_graph

In [7]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_graph ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   605.7    662.6   -289.9    579.7      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0994 -0.3173 -0.1006  0.3446  2.8043 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.38    3.222   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -6.0676     2.3957  -2.533  0.01132 *  
variableConcentration       0.7346     0.4053   1.812  0.06994 .  
variableSeparation         -0.1027     0.3733  -0.275  0.78319    
variableWidth             